### Load dependencies


In [ ]:
%pip install -q -r requirements.txt

### Load environment variables


In [ ]:
import os
from dotenv import load_dotenv

load_dotenv()

### Set up vector store


In [ ]:
import json
from langchain_openai import AzureOpenAIEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.documents import Document

embeddings = AzureOpenAIEmbeddings(
    api_key=os.environ["OPENAI_API_KEY"],
    azure_endpoint=os.environ["OPENAI_API_ENDPOINT"],
    deployment=os.environ["OPENAI_API_EMBEDDINGS_DEPLOYMENT"],
)

WORK_ORDERS = [
    {
        "issue": "Uneven pressure distribution",
        "solution": "Ensure the mold and platens are clean and properly aligned. Regularly calibrate the machine to maintain even pressure. Check for and replace any worn or damaged seals.",
        "date": "2024-01-12",
        "ID": "ISSUE-1001",
    },
    {
        "issue": "Incomplete curing",
        "solution": "Verify that the heating elements are functioning correctly and uniformly. Adjust the curing time and temperature settings according to material specifications. Regularly inspect and maintain heating elements and thermocouples.",
        "date": "2024-02-05",
        "ID": "ISSUE-1002",
    },
    {
        "issue": "Material flash",
        "solution": "Check mold clamping force and ensure it is adequate. Inspect the mold for wear and tear and repair or replace if necessary. Ensure proper material loading to prevent excess material overflow.",
        "date": "2024-01-28",
        "ID": "ISSUE-1003",
    },
    {
        "issue": "Air entrapment",
        "solution": "Implement proper venting techniques in the mold design. Use vacuum-assisted molding if necessary. Ensure the material is loaded correctly to minimize air pockets.",
        "date": "2024-02-16",
        "ID": "ISSUE-1004",
    },
    {
        "issue": "Sticking of parts to mold",
        "solution": "Apply an appropriate mold release agent before each cycle. Inspect and clean the mold surfaces regularly. Consider using molds with better release properties or modifying the mold design to reduce sticking.",
        "date": "2024-03-03",
        "ID": "ISSUE-1005",
    },
    {
        "issue": "Temperature fluctuations",
        "solution": "Check and maintain the thermostat and heating elements. Ensure proper insulation around the machine. Regularly monitor and adjust temperature settings to maintain consistency.",
        "date": "2024-01-14",
        "ID": "ISSUE-1006",
    },
    {
        "issue": "Hydraulic system leaks",
        "solution": "Inspect hydraulic lines and connections for any signs of wear or damage. Replace faulty components and tighten connections. Regularly check hydraulic fluid levels and top up as necessary.",
        "date": "2024-02-07",
        "ID": "ISSUE-1007",
    },
    {
        "issue": "Mold misalignment",
        "solution": "Ensure proper alignment of the mold before each cycle. Use alignment tools and jigs as necessary. Regularly inspect and calibrate the machine to maintain alignment accuracy.",
        "date": "2024-01-25",
        "ID": "ISSUE-1008",
    },
    {
        "issue": "Excessive cycle time",
        "solution": "Review and optimize the molding process parameters. Identify and eliminate any bottlenecks in the cycle. Regularly maintain and lubricate moving parts to ensure smooth operation.",
        "date": "2024-02-12",
        "ID": "ISSUE-1009",
    },
    {
        "issue": "Inconsistent part quality",
        "solution": "Ensure uniform material loading and distribution. Maintain consistent temperature and pressure throughout the cycle. Regularly inspect and clean the mold to prevent contamination.",
        "date": "2024-03-01",
        "ID": "ISSUE-1010",
    },
    {
        "issue": "Overheating of components",
        "solution": "Ensure proper ventilation and cooling around the machine. Regularly clean and maintain cooling systems. Monitor temperature levels and adjust settings to prevent overheating.",
        "date": "2024-01-30",
        "ID": "ISSUE-1011",
    },
    {
        "issue": "Control system errors",
        "solution": "Check for software updates and install as necessary. Inspect wiring and connections for any faults. Reset the control system and reconfigure settings if needed.",
        "date": "2024-02-10",
        "ID": "ISSUE-1012",
    },
    {
        "issue": "Mold surface damage",
        "solution": "Regularly inspect and repair any damage to the mold surface. Use protective coatings to prevent further damage. Replace the mold if necessary to ensure product quality.",
        "date": "2024-01-18",
        "ID": "ISSUE-1013",
    },
    {
        "issue": "Insufficient clamping force",
        "solution": "Check and adjust the clamping force settings. Inspect the clamping mechanism for wear or damage. Replace any faulty components to maintain adequate clamping force.",
        "date": "2024-02-20",
        "ID": "ISSUE-1014",
    },
    {
        "issue": "Electrical faults",
        "solution": "Inspect and test electrical components and wiring. Replace any faulty components and ensure proper grounding. Regularly maintain and check the electrical system to prevent faults.",
        "date": "2024-01-22",
        "ID": "ISSUE-1015",
    },
    {
        "issue": "Inconsistent material flow",
        "solution": "Ensure proper material preheating and mixing. Check for blockages or obstructions in the material feed system. Adjust material feed settings to maintain consistent flow.",
        "date": "2024-02-15",
        "ID": "ISSUE-1016",
    },
    {
        "issue": "Excessive mold wear",
        "solution": "Use high-quality mold materials and coatings. Regularly inspect and maintain the mold. Implement a preventive maintenance schedule to extend mold life.",
        "date": "2024-01-27",
        "ID": "ISSUE-1017",
    },
    {
        "issue": "Machine vibration",
        "solution": "Ensure the machine is properly leveled and secured. Inspect and tighten any loose components. Use vibration dampening materials and techniques to reduce vibration.",
        "date": "2024-02-18",
        "ID": "ISSUE-1018",
    },
    {
        "issue": "Cooling system failures",
        "solution": "Regularly inspect and maintain the cooling system components. Ensure proper coolant levels and flow. Replace any faulty cooling system parts to prevent failures.",
        "date": "2024-01-20",
        "ID": "ISSUE-1019",
    },
]

documents = [Document(page_content=json.dumps(wo)) for wo in WORK_ORDERS]

db = FAISS.from_documents(documents, embeddings)

### Import shared dependencies


In [ ]:
from langchain.tools import tool
from pydantic import BaseModel, Field
from typing import List, Optional

### Create query tools


In [ ]:
class WorkOrderSearch(BaseModel):
    issue: str = Field(
        ...,
        description="Issue reported by the maintenance tech to search work orders. For example, air entrapment or incomplete curing",
    )
    temporal: str = Field(
        ...,
        description="Temporal string that refers to the first or last `n` work orders. Options can be 'first', 'last' or 'all'",
    )
    n: Optional[int] = Field(..., description="Number of work orders requested")


@tool(args_schema=WorkOrderSearch)
def search_work_orders(issue: str, temporal: str, n: Optional[int] = 3) -> List:
    """Search for work orders where a similar issue was observed and/or resolved."""

    embedding = embeddings.embed_query(issue)
    results = db.similarity_search_with_score_by_vector(embedding, n)
    result_documents = [json.loads(result[0].page_content) for result in results]
    
    if temporal == "last":
        result_documents = sorted(result_documents, key=lambda doc: doc["date"], reverse=True)
    elif temporal == "first":
        result_documents = sorted(result_documents, key=lambda doc: doc["date"], reverse=False)
  
    return result_documents

In [ ]:
import pandas as pd
from pandasql import sqldf


class PartsSearch(BaseModel):
    partName: Optional[str] = (
        Field(..., description="Name or description of the part"),
    )
    partNumber: Optional[str] = Field(..., description="Part number or ID")


PARTS = [
    {
        "partName": "Heating Element",
        "partNumber": "HE-12345",
        "inventory": 15,
        "location": "Aisle 3, Shelf B",
    },
    {
        "partName": "Platen",
        "partNumber": "PL-67890",
        "inventory": 8,
        "location": "Aisle 1, Shelf D",
    },
    {
        "partName": "Hydraulic Cylinder",
        "partNumber": "HC-11223",
        "inventory": 5,
        "location": "Aisle 2, Shelf A",
    },
    {
        "partName": "Control Panel",
        "partNumber": "CP-44556",
        "inventory": 3,
        "location": "Aisle 4, Shelf C",
    },
    {
        "partName": "Thermocouple",
        "partNumber": "TC-78901",
        "inventory": 20,
        "location": "Aisle 3, Shelf F",
    },
]


@tool(args_schema=PartsSearch)
def search_parts(partName: Optional[str], partNumber: Optional[str]) -> List:
    """Search for a part, including its current inventory and location."""

    df = pd.read_json(json.dumps(PARTS))

    if partNumber is not None:
        return sqldf(f"SELECT * FROM df WHERE partNumber = '{partNumber}'", locals())
    elif partName is not None:
        return sqldf(f"SELECT * FROM df WHERE partName = '{partName}'", locals())
    else:
        return []

### Set up `AgentExecutor` to query data using tools


In [ ]:
from langchain.memory import ConversationBufferMemory
from langchain.agents import AgentExecutor
from langchain.schema.runnable import RunnablePassthrough
from langchain.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.utils.function_calling import convert_to_openai_function
from langchain.agents.output_parsers import OpenAIFunctionsAgentOutputParser
from langchain.agents.format_scratchpad import format_to_openai_functions
from langchain_openai import AzureChatOpenAI

tools = [search_parts, search_work_orders]
functions = [convert_to_openai_function(f) for f in tools]
model = AzureChatOpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    api_version="2024-02-01",
    azure_deployment=os.environ["OPENAI_API_DEPLOYMENT"],
    azure_endpoint=os.environ["OPENAI_API_ENDPOINT"],
    temperature=0,
).bind(functions=functions)
memory = ConversationBufferMemory(return_messages=True, memory_key="chat_history")
system_prompt = """You are a helpful manufacturing assistant that has access to multiple data sources at a manufacturing facility.
ONLY answer questions relevant in a manufacturing context.
Answer user questions using ONLY the provided context. Do not make up any information, and if you cannot answer the question using the context, say you don't know.
If the answer to the question can be found in chat history, simply answer the question and do not make any function calls. Otherwise, use a function call to retrieve data."""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        MessagesPlaceholder(variable_name="chat_history"),
        ("user", "{input}"),
        MessagesPlaceholder(variable_name="agent_scratchpad"),
    ]
)
chain = (
    RunnablePassthrough.assign(
        agent_scratchpad=lambda x: format_to_openai_functions(x["intermediate_steps"])
    )
    | prompt
    | model
    | OpenAIFunctionsAgentOutputParser()
)

qa = AgentExecutor(agent=chain, tools=tools, verbose=True, memory=memory)

### Chat with conversational agent


In [ ]:
memory.clear()

qa.invoke(
    {
        "input": "What are the last 2 work orders that address mold issues?"
    }
)

qa.invoke({"input": "What were the solutions of those work orders?"})

qa.invoke({"input": "What is the current inventory of the Hydraulic Cylinder part?"})